# Работа с API


In [ ]:
import requests as rq
import json
public = rq.get('http://marvel.task-sss.krasilnikov.spb.ru/api/authorize?public_key=dd2bdf6cb881176bc2c0c568e1a180fa').json()
private_key = public['private_key']
private_key

'HQLXO4RDT3HB0UP65E3ZZY32M2K6KPLIF45MSDQV'

In [ ]:
import hashlib
ts = 'Wed Jan 1 00:00:00 2020'
public_key = 'dd2bdf6cb881176bc2c0c568e1a180fa'
p = ts+private_key+public_key
eg_hash = hashlib.blake2b(p.encode()).hexdigest()
eg_hash

'5611253161ce041010e0aeb16ed1a5c086c70fad5a3f49630b13b9bb46f12d8fab684d4395421fe6f0895d78b29c68776bf4458f093009adf29f3e6239a69b7d'

Загрузка информации по всем персонажам комиксов: Идентификатор, Имя, Описание

In [ ]:
dostup = 'public_key='+public_key+'&ts='+ts+'&hash='+eg_hash
data = rq.get('http://marvel.task-sss.krasilnikov.spb.ru/api/characters?'+dostup).json()
import pandas as pd
k =[]
for i in data['data']['results']:
    for j in i:
        k.append({'id':j['id'], 'name': j['name'], 'description':j['description']})
all_characters = pd.DataFrame(k)
all_characters['id'] = all_characters['id'].astype('int64')
all_characters = all_characters.sort_values(by=['id'])
all_characters

,id,name,description
0,1009144,A.I.M.,AIM is a terrorist organization bent on destro...
1,1009153,Angel (Warren Worthington III),
2,1009154,Annihilus,
3,1009159,Archangel,
4,1009163,Aurora,
...,...,...,...
211,1017574,Angela (Aldrif Odinsdottir),
212,1017575,Captain America (Sam Wilson),"<p class=""Body"">Sam Wilson, a Harlem native, p..."
213,1017576,Thor (Goddess of Thunder),"<p class=""Body"">When the Odinson lost his abil..."
214,1017577,Ms. Marvel (Kamala Khan),A Muslim-American teenager growing up in Jerse...


Создание датасета с информацией о комиксах автора Kelly Thompson


In [ ]:
data1 = rq.get('http://marvel.task-sss.krasilnikov.spb.ru/api/creators?name=Kelly Thompson&'+dostup).json()
spisok = data1 ['data']['results'][0]['comics']['items']
comics= pd.DataFrame(spisok)
comics.rename(columns={'resourceID':'id', 'name':'title'}, inplace = True)
id_list = list(comics['id'])
s = []
for i in id_list:
    a = rq.get('http://marvel.task-sss.krasilnikov.spb.ru/api/comics?id='+i+'&'+dostup).json()
    s.append({'id':a['data']['results'][0]['id'], 'pages_count':a['data']['results'][0]['pageCount'],
              'creators_count': a['data']['results'][0]['creators']['available'],
              'characters_count':a['data']['results'][0]['characters']['available']})
p = pd.DataFrame(s)
comics['id'] = comics['id'].astype('int64')
p['id'] = p['id'].astype('int64')
comics = comics.join(p.set_index('id'), on = 'id')
comics = comics.sort_values(by='id').reset_index(drop=True)
comics

,id,title,pages_count,creators_count,characters_count
0,55896,A-Force Vol. 1: Hypertime (Trade Paperback),144,4,0
1,55899,A-Force Vol. 2: Rage Against the Dying of the ...,112,4,0
2,56013,A-Force (2016) #2,32,3,0
3,56014,A-Force (2016) #3,32,5,0
4,56015,A-Force (2016) #4,32,4,0
5,56016,A-Force (2016) #5,32,3,0
6,56017,A-Force (2016) #6,32,3,5
7,56018,A-Force (2016) #7,32,3,5
8,56019,A-Force (2016) #8,32,3,5
9,56020,A-Force (2016) #9,32,3,5


Создание датафрейм **characters**, включающий в себя информацию о доступных персонажах из всех комиксов автора Kelly Thompson, созданный на предыдущем этапе.

 Для каждого персонажа сохраняется следующая информация: общее количество _доступных_ Комиксов с участием этого персонажа (НЕ количество автоматически выданных при запросе комиксов), общее количество _доступных_ Серий, Историй и Событий с участием этого персонажа, Дата последнего изменения данных.

In [ ]:
t=[]
for i in id_list:
    data2 = rq.get('http://marvel.task-sss.krasilnikov.spb.ru/api/comics?id='+i+'&'+dostup).json()
    if data2['data']['results'][0]['characters']['items'] == []:
        continue
    elif data2['data']['results'][0]['characters']['items'] not in t:
        for j in range(len(data2['data']['results'][0]['characters']['items'])):
            if data2['data']['results'][0]['characters']['items'][j]not in t:
                t.append(data2['data']['results'][0]['characters']['items'][j])
characters = pd.DataFrame(t)
characters = characters.rename(columns={'resourceID':'id'})
row = []
for k in list(characters['id']):
    query = rq.get('http://marvel.task-sss.krasilnikov.spb.ru/api/characters?id='+k+'&'+dostup).json()
    row.append({'id':k, 'comics_count': query['data']['results'][0]['comics']['available'],
           'series_count': query['data']['results'][0]['series']['available'],
           'stories_count': query['data']['results'][0]['series']['available'],
           'events_count': query['data']['results'][0]['events']['available'],
            'last_modified': query['data']['results'][0]['modified']})
row = pd.DataFrame(row)
row['id'] = row['id'].astype('int64')
characters['id'] = characters['id'].astype('int64')
characters = characters.join(row.set_index('id'), on ='id')
characters['comics_count'] = characters['comics_count'].astype('int64')
characters = characters.sort_values(by='comics_count', ascending = False).reset_index(drop=True)
characters

,id,name,comics_count,series_count,stories_count,events_count,last_modified
0,1009610,Spider-Man,3947,1002,1002,38,2020-07-21T10:30:10-0400
1,1010338,Captain Marvel (Carol Danvers),512,177,177,16,2020-07-21T10:33:02-0400
2,1009583,She-Hulk (Jennifer Walters),453,148,148,15,2014-01-24T18:10:30-0500
3,1009267,Dazzler,205,68,68,11,2012-06-21T14:16:22-0400
4,1009438,Medusa,164,78,78,10,2014-06-18T13:02:36-0400
5,1010701,Elsa Bloodstone,35,16,16,2,1969-12-31T19:00:00-0500
6,1010746,Nico Minoru,21,9,9,1,1969-12-31T19:00:00-0500
